In [15]:
import sqlite3
import json
from gensim.corpora import Dictionary
from gensim.models.ldamodel import LdaModel
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re

In [2]:
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

query = "SELECT transcript FROM podcast_episodes"
cursor.execute(query)
episodes = cursor.fetchall()

print(f"There are {len(episodes)} episode transcripts in the sample.")
conn.commit()
conn.close()

There are 10003 episode transcripts in the sample.


Replicating LDA with parameters used in paper

In [49]:
# Load stopwords
stop_words = set(stopwords.words("english"))

# Preprocessing function
def preprocess_text(text):
    # Tokenize and clean text
    tokens = word_tokenize(text.lower())
    tokens = [re.sub(r"[^a-z]", "", token) for token in tokens]  # Keep only alphabets
    tokens = [token for token in tokens if token and token not in stop_words and len(token) > 2]
    return tokens

# Connect to database
db_path = "data.db"
conn = sqlite3.connect(db_path)

# Fetch all records from the table
cursor = conn.cursor()
cursor.execute("SELECT rowid, transcript FROM podcast_episodes")
rows = cursor.fetchall()

# Process transcripts to extract the first 1000 words
processed_data = []
texts = []
id_mapping = []

for row in rows:
    episode_id, transcript = row
    if transcript and isinstance(transcript, str) and transcript.strip():
        try:
            words = preprocess_text(" ".join(word_tokenize(transcript)[:1000]))
            transcript_1000 = " ".join(words)
        except Exception as e:
            print(f"Tokenization error for episode ID {episode_id}: {e}")
            transcript_1000 = ""
    else:
        transcript_1000 = ""
    
    processed_data.append((transcript_1000, episode_id))
    texts.append(words if transcript_1000 else [])
    id_mapping.append(episode_id)

# Update transcript_1000 in the database
cursor.executemany("UPDATE podcast_episodes SET transcript_1000_tokens = ? WHERE rowid = ?", processed_data)
conn.commit()
conn.close()

# Build dictionary and corpus for LDA
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

Tokenization error for episode ID 6660: list index out of range


In [50]:
# Connect to database
db_path = "data.db"
conn = sqlite3.connect(db_path)

# Fetch all records from the table
cursor = conn.cursor()
cursor.execute("SELECT rowid FROM podcast_episodes")
rows = cursor.fetchall()

# Set Gensim LDA parameters similar to MALLET
num_topics = 200
passes = 10  # Rough equivalent to optimize_interval
workers = 50  # Multi-threading for parallel processing

# Train LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)

# Prepare topic data for each episode
update_data = []
for i, bow in enumerate(corpus):
    episode_id = id_mapping[i]
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0)
    dominant_topic = max(topic_dist, key=lambda x: x[1])[0] if topic_dist else None

    # Convert probabilities to standard Python float
    dist_dict = {int(topic_id): float(prob) for topic_id, prob in topic_dist}
    
    # JSON serialization
    json_distribution = json.dumps(dist_dict)
    
    update_data.append((dominant_topic, json_distribution, episode_id))

# Update the database with dominant_topic_number and topic_distribution
cursor.executemany("UPDATE podcast_episodes SET dominant_topic_number_1000 = ?, topic_distribution_1000 = ? WHERE rowid = ?", update_data)
conn.commit()
conn.close()

print("Database updated with topic modeling 1000 results.")

# Save outputs similar to MALLET
lda_model.save("gensim_lda_model")
topics = lda_model.print_topics(num_topics=num_topics)
with open("../data/doc_topics_1000.txt", "w") as doc_topics_file:
    for doc_num, topic_probs in enumerate(lda_model[corpus]):
        doc_topics_file.write(f"Document {doc_num}: {topic_probs}\n")

with open("../data/topic_keys_1000.txt", "w") as topic_keys_file:
    for topic_num, topic_words in topics:
        topic_keys_file.write(f"Topic {topic_num}: {topic_words}\n")

Database updated with topic modeling 1000 results.


Whole transcript tokens

In [43]:
# Connect to database
db_path = "data.db"
conn = sqlite3.connect(db_path)

# Fetch all records from the table
cursor = conn.cursor()
cursor.execute("SELECT rowid, transcript FROM podcast_episodes")
rows = cursor.fetchall()

# Process transcripts to extract the first 1000 words
processed_data = []
texts = []
id_mapping = []

for row in rows:
    episode_id, transcript = row
    if transcript and isinstance(transcript, str) and transcript.strip():
        try:
            words = preprocess_text(" ".join(word_tokenize(transcript)))
            transcript = " ".join(words)
        except Exception as e:
            print(f"Tokenization error for episode ID {episode_id}: {e}")
            transcript = ""
    else:
        transcript = ""
    
    processed_data.append((transcript, episode_id))
    texts.append(words if transcript else [])
    id_mapping.append(episode_id)

# Update transcript_1000 in the database
cursor.executemany("UPDATE podcast_episodes SET transcript_tokens = ? WHERE rowid = ?", processed_data)
conn.commit()
conn.close()

# Build dictionary and corpus for LDA
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

Tokenization error for episode ID 6660: list index out of range


In [52]:
# Connect to database
db_path = "data.db"
conn = sqlite3.connect(db_path)

# Fetch all records from the table
cursor = conn.cursor()
cursor.execute("SELECT rowid FROM podcast_episodes")
rows = cursor.fetchall()

# Set Gensim LDA parameters similar to MALLET
num_topics = 200
passes = 10  # Rough equivalent to optimize_interval
workers = 50  # Multi-threading for parallel processing

# Train LDA model
lda_model = LdaModel(corpus, num_topics=num_topics, id2word=dictionary, passes=10)
# num_topics = 200
# lda_model = LdaModel(
#     corpus=corpus,
#     id2word=dictionary,
#     num_topics=num_topics,
#     passes=passes,  # Number of passes over the entire corpus (analogous to optimize interval)
#     iterations=100,  # Optional: Set higher for better convergence
#     random_state=42,
#     chunksize=1000,
#     alpha='auto',  # Automatic hyperparameter optimization
#     eta='auto',
#     eval_every=None  # Turn off frequent model evaluation for efficiency
# )
# Prepare topic data for each episode
update_data = []
for i, bow in enumerate(corpus):
    episode_id = id_mapping[i]
    topic_dist = lda_model.get_document_topics(bow, minimum_probability=0)
    dominant_topic = max(topic_dist, key=lambda x: x[1])[0] if topic_dist else None

    # Convert probabilities to standard Python float
    dist_dict = {int(topic_id): float(prob) for topic_id, prob in topic_dist}
    
    # JSON serialization
    json_distribution = json.dumps(dist_dict)
    
    update_data.append((dominant_topic, json_distribution, episode_id))

# Update the database with dominant_topic_number and topic_distribution
cursor.executemany("UPDATE podcast_episodes SET dominant_topic_number = ?, topic_distribution = ? WHERE rowid = ?", update_data)
conn.commit()
conn.close()

print("Database updated with topic modeling results.")

# Save outputs similar to MALLET
lda_model.save("gensim_lda_model")
topics = lda_model.print_topics(num_topics=num_topics)
with open("../data/doc_topics.txt", "w") as doc_topics_file:
    for doc_num, topic_probs in enumerate(lda_model[corpus]):
        doc_topics_file.write(f"Document {doc_num}: {topic_probs}\n")

with open("../data/topic_keys.txt", "w") as topic_keys_file:
    for topic_num, topic_words in topics:
        topic_keys_file.write(f"Topic {topic_num}: {topic_words}\n")

Database updated with topic modeling results.


Using MALLET

In [ ]:
# import sqlite3
# import json
# from gensim.models.wrappers import LdaMallet
# from gensim.corpora import Dictionary
# from gensim.models import CoherenceModel
# import os

# # Path to MALLET binary (update this path as necessary)
# path_to_mallet_binary = "~/Documents/GitHub/Mallet/"

# # Database connection setup
# db_path = "data.db"
# conn = sqlite3.connect(db_path)
# cursor = conn.cursor()

# # Fetch text data from the database
# cursor.execute("SELECT rowid, text_column FROM podcast_episodes")  # Replace text_column with your column name
# rows = cursor.fetchall()

# # Prepare data for LDA
# id_mapping = {}
# documents = []
# for rowid, text in rows:
#     id_mapping[len(documents)] = rowid
#     documents.append(text.split())  # Tokenize text; adjust as needed based on preprocessing

# # Create a Gensim dictionary and corpus
# dictionary = Dictionary(documents)
# corpus = [dictionary.doc2bow(doc) for doc in documents]

# # Define LDA parameters
# num_topics = 200
# passes = 10

# # Train LDA Mallet model
# lda_mallet = LdaMallet(
#     mallet_path,
#     corpus=corpus,
#     num_topics=num_topics,
#     id2word=dictionary,
#     optimize_interval=passes
# )

# # Extract topics for each document
# update_data = []
# for i, bow in enumerate(corpus):
#     episode_id = id_mapping[i]
#     topic_dist = lda_mallet[bow]
#     dominant_topic = max(topic_dist, key=lambda x: x[1])[0] if topic_dist else None

#     # Convert probabilities to standard Python float
#     dist_dict = {int(topic_id): float(prob) for topic_id, prob in topic_dist}

#     # JSON serialization
#     json_distribution = json.dumps(dist_dict)

#     update_data.append((dominant_topic, json_distribution, episode_id))

# # Update the database
# cursor.executemany("UPDATE podcast_episodes SET dominant_topic_number_mallet = ?, topic_distribution_mallet = ? WHERE rowid = ?", update_data)
# conn.commit()
# conn.close()

# print("Database updated with topic modeling results.")

# # Save model and topic outputs
# lda_mallet.save("gensim_lda_mallet_model")

# # Save document topics
# doc_topics_path = "../data/doc_topic_mallet.txt"
# with open(doc_topics_path, "w") as doc_topics_file:
#     for doc_num, topic_probs in enumerate(lda_mallet[corpus]):
#         doc_topics_file.write(f"Document {doc_num}: {topic_probs}\n")

# # Save topic keys
# topic_keys_path = "../data/topic_keys_mallet.txt"
# topics = lda_mallet.show_topics(num_topics=num_topics, formatted=True)
# with open(topic_keys_path, "w") as topic_keys_file:
#     for topic_num, topic_words in topics:
#         topic_keys_file.write(f"Topic {topic_num}: {topic_words}\n")

# print("Topic modeling outputs saved.")

Check racial justice topics episode count

In [51]:
# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the query to count the number of rows with dominant_topic_number = ?
query = "SELECT COUNT(*) FROM podcast_episodes WHERE dominant_topic_number_1000 = 6 OR dominant_topic_number_1000 = 110"
cursor.execute(query)

# Fetch the result
count = cursor.fetchone()[0]
print(f"Number of rows with racial justice-related dominant_topic_number_1000: {count}")

# Close the cursor and the connection
cursor.close()
conn.close()

Number of rows with racial justice-related dominant_topic_number_1000: 127


In [63]:
# Connect to the database
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Execute the query to get column names
query = 'PRAGMA table_info(podcast_episodes)'
cursor.execute(query)

# Fetch the result
columns_info = cursor.fetchall()
column_names = [info[1] for info in columns_info]
print(f"Column names in podcast_episodes: {column_names}")

# Close the cursor and the connection
cursor.close()
conn.close()

Column names in podcast_episodes: ['transcript', 'rssUrl', 'epTitle', 'epDescription', 'mp3url', 'podTitle', 'lastUpdate', 'itunesAuthor', 'itunesOwnerName', 'explicit', 'imageUrl', 'language', 'createdOn', 'host', 'podDescription', 'category1', 'category2', 'category3', 'category4', 'category5', 'category6', 'category7', 'category8', 'category9', 'category10', 'oldestEpisodeDate', 'episodeDateLocalized', 'durationSeconds', 'hostPredictedNames', 'numUniqueHosts', 'guestPredictedNames', 'numUniqueGuests', 'neitherPredictedNames', 'numUniqueNeithers', 'mainEpSpeakers', 'numMainSpeakers', 'hostSpeakerLabels', 'guestSpeakerLabels', 'overlapPropTurnCount', 'avgTurnDuration', 'overlapPropDuration', 'totalSpLabels', 'BLM', 'description_tokens', 'description_topic', 'transcript_tokens', 'transcript_topic', 'sentences', 'topic_distribution', 'transcript_1000_tokens', 'dominant_topic_number', 'dominant_topic_number_1000', 'topic_distribution_1000']
